In [1]:
# https://research.samsung.com/

In [5]:
import json
import os 
import requests
import time
from datetime import datetime,timedelta
from datetime import date
import re
import sys
import urllib, urllib.request, urllib.parse
import random
from scrawl import *

    
# Date and time
start_time = time.time()
current_time = datetime.now().strftime("%H-%M-%S")
created_on = date.today().strftime("%Y-%m-%d")

# client_id = sys.argv[1]
client_id = '5f69d22ef472d6646f577fa6'  # Europe
site = 'research_samsung_com_publication'
c = Crawl()  # creating object
site_name = 'Samsung AI Research Center (Republic of Korea)'

# Driver Connection      
# options = webdriver.ChromeOptions()
# options.add_argument('--headless')
# browser = webdriver.Chrome(f'{project_path}/chromedriver', options = options )
# browser.maximize_window()

#firefox
# options = Options()
# options.headless = True
# browser = webdriver.Firefox(options=options)
# browser.maximize_window()

# create directories to store logs.
log_path = c.create_directories(project_path, client_id, site)

## creating pdf directories
pdf_directory = c.create_pdf_directories(project_path, site)
# create image directories
image_directory = c.create_image_directories(project_path)

# logger
logger = log_func(log_path, created_on, current_time)
logger.info("Process Started ...\n")

# initialize variables
skipped_due_to_headline = 0
skipped_due_to_content = 0
skipped_due_to_date = 0
missing_overall_tonality = 0
no_of_data = 0
duplicate_data = 0  
unable_to_fetch_article_url = 0
unable_to_fetch_cat_url = 0
unable_to_download_pdf = 0
publish_source = 'research.samsung.com'
country = 'South Korea'
language = 'English'
images_path = []

foot_fall = c.get_foot_fall(publish_source)

home_page = c.download_page('https://research.samsung.com')

urls =['https://research.samsung.com/artificial-intelligence',
'https://research.samsung.com/next-generation-communications',
'https://research.samsung.com/robot']

for url in urls:  
    cat_url = url  
    logger.info(f'Fetching cat url  {cat_url}\n')
    print("cat_url:",cat_url)
#     continue
    cat_page = c.download_page(cat_url)    
    if cat_page.startswith('Unable to fetch'):
        logger.info(cat_page) 
        unable_to_fetch_cat_url += 1
        continue 

    cat_page = c.scrap('<ul\s*class="articles-inner"(.*?)<footer\s*id="footer">',cat_page)
    for i in cat_page.split('<li')[1:5]:
        # source_link
        source_link = c.scrap('href="(.*?)"', i)
        print(source_link)
#         continue

        # handle duplicates
        source_link_query = {'source_link':source_link}
        dic = cl_data.find_one(source_link_query,{'source_link': 1}) 
        if dic:
            duplicate_data += 1
            continue

        time.sleep(random.randint(1,3))

        logger.info(f'Fetching {source_link}\n')
        
        page = c.download_page(source_link)   # here the page wil get download
        
        if page.startswith('Unable to fetch'):     
            logger.info(page) # writes error message with error code
            unable_to_fetch_article_url += 1
            continue   
        if 'blog' in source_link:
            source_headline = c.scrap('/blog/(.*)',source_link)
        else:
            source_headline = c.scrap('<title>(.*?)</title>', page)
            if not source_headline:
                 source_headline = c.scrap('<h1.*?>(.*?)</h1>', page)  
        source_headline = re.sub('&.*?;','',source_headline)
        print(source_headline)

       
        # skip if headline not found
        if not source_headline:
            logger.info(f'Skipping due to headline {source_link}\n')
            skipped_due_to_headline += 1
            continue
            
         # Date and time
        pub_date, publish_time = '', ''
        publish_time = current_time
        try:
            date_time_str = c.scrap('class="articles-date".*?>On(.*?)<', i)
            date_time_str = re.sub('[^\w+]', '', date_time_str)  
            date_time_obj = datetime.strptime(date_time_str, '%B%d%Y')  
            ist_date_time = date_time_obj + timedelta(hours = 0,minutes = 0)  
            ist_date_time = ist_date_time.strftime('%Y-%m-%d %H:%M:%S')
            pub_date = ist_date_time[:10]
            publish_time = ist_date_time[11:]
            print(pub_date)
            
        except:
            pass

        # skip null date
        if not pub_date:
            logger.info(f'Skipping due to date {source_link}\n')            
            skipped_due_to_date += 1
            continue

        # break if date is not today's date
#         if pub_date != created_on:
#             break    


        # source_content         
        source_content= c.scrap('<div\s*class="news-con"><p>(.*?)<footer\s*id="footer">',page)   
        if not source_content:
            source_content= c.scrap('div\s*class="text_cont">(.*?)<div\s*class="fxb-section">',page)#<div class="top_area clearfix">
        if not source_content:
            source_content= c.scrap('<div\s*class="news-con">(.*?)<footer\s*id="footer">',page)# <div class="top_area clearfix">
        if not source_content:
            source_content= c.scrap('<div\s*class="news-con">(.*?)<div\s*class="top_area clearfix">',page)
        source_content = re.sub('<script>(.*?)</script>','',source_content)
        source_content = re.sub('&.*?;','',source_content)
        source_content = c.strip_html(source_content)  
        if not source_content:
            logger.info(f'Skipping due to content {source_link}\n')            
            skipped_due_to_content += 1
            continue
        print(source_content)
        continue
        
        journalist =c.scrap("'author-name':'(.*?)'",page)

        if not journalist: journalist = 'NA'


        # current date and time 00
        harvest_time = datetime.now().strftime("%H:%M:%S")

        # temp link
        temp_link = source_link

        # headline and content 
        headline = source_headline
        content = source_content

        # overall_tonality
        overall_tonality = ''

        # word count
        word_count = len((source_headline + ' ' + source_content).split())

        html_content = ''

        # image_urls
        image_urls = []
    

        # storing the above data in a dictionary
        clientdata ={
                        "client_master" : client_id, 
                        "articleid":client_id,
                        "medium":'Web' ,
                        "searchkeyword":[],
                        "entityname" : [] ,
                        "process_flage":"1",
                        "na_flage":"0",
                        "na_reason":"",
                        "qc_by":"",
                        "qc_on":"",
                        "location":"",
                        "spokeperson":"",
                        "quota":"",
                        "overall_topics":"",
                        "person":"",
                        "overall_entites":"",
                        "overall_tonality": overall_tonality,
                        "overall_wordcount":word_count,
                        "article_subjectivity":"",
                        "article_summary":"",
                        "pub_date":pub_date,
                        "publish_time":publish_time,
                        "harvest_time":harvest_time,
                        "temp_link":temp_link,
                        "publish_source": publish_source,
                        "programme":'null',
                        "feed_class":"News",
                        "publishing_platform":"",
                        "klout_score":"",
                        "journalist":journalist,
                        "headline":headline,
                        "content":content,
                        "source_headline":source_headline,
                        "source_content":source_content,
                        "language":language,
                        "presence":'null',
                        "clip_type":'null',
                        "prog_slot":'null',
                        "op_ed":'0',
                        "location_mention":'',
                        "source_link":source_link,
                        "author_contact":'',
                        "author_emailid":'',
                        "author_url":'',
                        "city":'',
                        "state":'',
                        "country":country,
                        "source":publish_source,
                        "foot_fall":foot_fall,
                        "created_on":created_on,
                        "active":'1',
                        'crawl_flag':2,
                        "images_path":images_path,
                        "html_content":html_content,
                        "pdf_url": pdf_url,
                        "pdf_name": pdf_name,
                        "pdf_path":pdf_path
                    }
#         cl_data.insert_one(clientdata)  
        no_of_data += 1
logger.info('Iteration complete\n')   
logger.info(f'Number of data: {no_of_data}\n')
logger.info(f'Duplicate data: {duplicate_data}\n')
logger.info(f'Unable to fetch cat url: {unable_to_fetch_cat_url}\n')
logger.info(f'Unable to fetch article url: {unable_to_fetch_article_url}\n')
logger.info(f'Skipped due to headline: {skipped_due_to_headline}\n')
logger.info(f'Skipped due to content: {skipped_due_to_content}\n')
logger.info(f'Skipped due to date: {skipped_due_to_date}\n')
logger.info(f'Unable to download pdf: {unable_to_download_pdf}\n')
logger.info(f'country: {country}\n')
logger.info(f'language: {language}\n')
logger.info(f'Processing finished in {time.time() - start_time} seconds.\n')

cat_url: https://research.samsung.com/artificial-intelligence
https://research.samsung.com/news/Open-Source-Project-for-MPEG-5-EVC-Essential-Video-Coding
Open-Source Project for MPEG-5 EVC (Essential Video Coding) | Samsung Research
2021-07-08
The COVID-19 outbreak has changed our lives and relationships over the past year. We now have no other choice but to communicate more often through the Internet. One of the key technologies for talking via the Internet is, definitely, video compression (coding) technology. However, a very old codec, H.264/AVC, is still the primary technology used in video conference applications, whose standardization was finalized about 20 years ago. Experts had undoubtedly developed a more efficient video codec called HEVC, but it is not dominantly available yet because of its complex license policy or computational complexity issues for video conferences.

In June 2021, Samsung Research released an open-source video encoder (XEVE: eXtra-fast Essential Video En

Learning-to-Align-Temporal-Sequences
2021-06-21
Introduction											
					Temporal sequences (e.g., videos) are an appealing data source as they provide a rich source of information and additional constraints to leverage in learning. By far the main focus on temporal sequence analysis in computer vision has been on learning representations (i.e., compact abstractions of the input data) targeting high-level distinctions between signals (e.g., action classification, “What action is present in the video?”).  Alternatively, there is work that seek a finer-grained understanding of sequences (e.g., “What particular phase of the action is performed”) but rely on fully-supervised methods, where labels are attached to each element of a sequence.  Acquiring such labels for training at large-scale is a highly laborious, expensive process.

					In this blog, we present our recent work [1] published at CVPR 2021, which reduces the amount of labeled training data needed for learning useful repre

https://research.samsung.com/news/Samsung-Researchers-elected-to-lead-UWB-Technical-Standards-at-FiRa-Consortium
Samsung Researchers elected to lead UWB Technical Standards at FiRa Consortium | Samsung Research
2021-07-07
Samsung Research is pleased to announce that several of its’ researchers have been elected as chairs to lead the next-generation ultra-wideband (UWB) standards for two-year term at FiRa Consortium.

Karthik Srinivasa Gopalan, at Samsung R&D Institute India - Bangalore, was re-elected as co-chair of the Technical Working Group, and he will continue leading to deliver technical specifications for the UWB interoperability. Karthik has more than 20 years of experience and has worked extensively in the areas of research, development and standardization in peer-to-peer, contents recommendations, UMTS, Wi-Fi and UWB systems.

Clint Chaplin, at Samsung Research America, was also re-elected as co-chair of the Compliance & Certification Working Group, and he will continue to le

Samsung Electronics and University of California Santa Barbara Demonstrate 6G Terahertz Wireless Communication Prototype  Samsung Global Newsroom
2021-06-16
https://research.samsung.com/blog/Open-Source-Kubernetes-for-the-next-level-of-network-virtualization
Open-Source-Kubernetes-for-the-next-level-of-network-virtualization
2021-05-27
We as Samsung Research have recently presented a research achievement as a global team work with Samsung R&D Institute Poland and Samsung Research America for improving the Kubernetes open source project into the next level of competency in IEEE INFOCOM 2021.

					What is a background of this research?
					Container technology has revolutionized the way software is being packaged and run. The telecommunications industry, now challenged with the 5G transformation, views containers as the best way to achieve agile infrastructure that offers high throughput and low latency for 5G edge applications. These challenges make optimal scheduling of performance-s

https://research.samsung.com/blog/Newest-ROS2-Distribution-Galactic-Geochelone-Released
Newest-ROS2-Distribution-Galactic-Geochelone-Released
2021-05-27
May 2021 comes with many new and exciting changes: easing of COVID-19 restrictions in parts of the world, a warm and eventful Kentucky Derby, and the newest release of ROS2, Galactic Geochelone! Galactic is the 7th release of the new Robot Operating System (ROS) 2.0, built on DDS to be the production grade middleware for robotics with all of the amazing tools you came to love in ROS.

					ROS2 was redesigned from the ground up to handle non-ideal networks found in real-world production environments, communicate between multiple robots in a fleet securely, operate on low-power embedded systems, and leverage the real-time capacity of several different DDS vendors. Galactic is another step forward in its journey; extending existing capabilities and expanding new ones!

					Released on May 23rd, Galactic provides many improvements that h